In [5]:
import numpy as np 
import pandas as pd 

from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
# There are three types of NB model.
from sklearn.datasets import fetch_covtype, fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn import metrics

In [3]:
prior = [0.45,0.3,0.15,0.1]
likelihood = [[0.3,0.3,0.4],[0.7,0.2,0.1],[0.15,0.5,0.35],[0.6,0.2,0.2]]

idx = 0 # This is indexing number.
for c,xs in zip(prior,likelihood): # Both objects have 4 elements.
    # zip function concatenate two different datasets.
    result = 1.0
    for x in xs:
        result *= x
    result *= c
    idx+=1
    print(f"{idx}th probability : {result}")

1th probability : 0.0162
2th probability : 0.0042
3th probability : 0.0039375
4th probability : 0.0024000000000000002


In [6]:
covtype = fetch_covtype() # we are fetching dataset from online.
# This data is related to the forest in the United States. 
# Classification problem (Multi-variate)
# Classifying the soil.
# Consists of 580000 samples.
print(covtype.DESCR)

.. _covtype_dataset:

Forest covertypes
-----------------

The samples in this dataset correspond to 30Ã—30m patches of forest in the US,
collected for the task of predicting each patch's cover type,
i.e. the dominant species of tree.
There are seven covertypes, making this a multiclass classification problem.
Each sample has 54 features, described on the
`dataset's homepage <https://archive.ics.uci.edu/ml/datasets/Covertype>`__.
Some of the features are boolean indicators,
while others are discrete or continuous measurements.

**Data Set Characteristics:**

    =================   ============
    Classes                        7
    Samples total             581012
    Dimensionality                54
    Features                     int
    =================   ============

:func:`sklearn.datasets.fetch_covtype` will load the covertype dataset;
it returns a dictionary-like 'Bunch' object
with the feature matrix in the ``data`` member
and the target values in ``target``. If optional 

In [16]:
pd.DataFrame(covtype.data)
covtype.target # This is the label of each sample data.

array([5, 5, 2, ..., 3, 3, 3])

In [20]:
# Splitting training and test data.
covtype_X = covtype.data
covtype_Y = covtype.target
covtype_Xtrain, covtype_Xtest, covtype_Ytrain, covtype_Ytest = train_test_split(covtype_X,covtype_Y,test_size=0.2)
print(f'size of data : {covtype_X.shape}')
print(f'size of training dataset = {covtype_Xtrain.shape}')
print(f'size of test dataset = {covtype_Xtest.shape}')

size of data : (581012, 54)
size of training dataset = (464809, 54)
size of test dataset = (116203, 54)


In [23]:
covtype_df = pd.DataFrame(data=covtype_X)
covtype_df.describe()
# describe function will provide statistical description of the data.

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
count,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,...,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000
mean,2959.365301,155.656807,14.103704,269.428217,46.418855,2350.146611,212.146049,223.318716,142.528263,1980.291226,...,0.044175,0.090392,0.077716,0.002773,0.003255,0.000205,0.000513,0.026803,0.023762,0.015060
std,279.984734,111.913721,7.488242,212.549356,58.295232,1559.254870,26.769889,19.768697,38.274529,1324.195210,...,0.205483,0.286743,0.267725,0.052584,0.056957,0.014310,0.022641,0.161508,0.152307,0.121791
min,1859.000000,0.000000,0.000000,0.000000,-173.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2809.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,1024.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2996.000000,127.000000,13.000000,218.000000,30.000000,1997.000000,218.000000,226.000000,143.000000,1710.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3163.000000,260.000000,18.000000,384.000000,69.000000,3328.000000,231.000000,237.000000,168.000000,2550.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3858.000000,360.000000,66.000000,1397.000000,601.000000,7117.000000,254.000000,254.000000,254.000000,7173.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [31]:
covtype_train_df = pd.DataFrame(data = covtype_Xtrain)
covtype_train_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
count,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,...,464809.000000,464809.000000,464809.000000,464809.00000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000
mean,2959.576794,155.685847,14.099916,269.337326,46.418809,2350.432653,212.170928,223.338010,142.521292,1979.992943,...,0.044063,0.090745,0.077701,0.00279,0.003281,0.000215,0.000447,0.026893,0.023730,0.015041
std,279.790905,111.867186,7.485995,212.415502,58.190296,1559.683211,26.776050,19.751468,38.302085,1324.642919,...,0.205236,0.287246,0.267700,0.05275,0.057185,0.014666,0.021149,0.161770,0.152207,0.121714
min,1859.000000,0.000000,0.000000,0.000000,-166.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2809.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,1024.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2996.000000,127.000000,13.000000,218.000000,30.000000,1998.000000,218.000000,226.000000,143.000000,1710.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3163.000000,260.000000,18.000000,384.000000,69.000000,3328.000000,231.000000,237.000000,168.000000,2550.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3858.000000,360.000000,66.000000,1390.000000,601.000000,7117.000000,254.000000,254.000000,254.000000,7173.000000,...,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [27]:
covtype_test_df = pd.DataFrame(data = covtype_Xtest)
covtype_test_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
count,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,...,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000
mean,2958.519333,155.540649,14.118852,269.791778,46.419043,2349.002453,212.046531,223.241543,142.556147,1981.484351,...,0.044620,0.088982,0.077778,0.002702,0.003150,0.000164,0.000775,0.026445,0.023889,0.015137
std,280.758321,112.100072,7.497234,213.084455,58.713347,1557.546520,26.745114,19.837360,38.164258,1322.407890,...,0.206469,0.284719,0.267823,0.051912,0.056034,0.012786,0.027819,0.160456,0.152705,0.122100
min,1866.000000,0.000000,0.000000,0.000000,-173.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2808.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,1025.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2995.000000,127.000000,13.000000,218.000000,29.000000,1995.000000,218.000000,226.000000,143.000000,1712.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3163.000000,261.000000,18.000000,390.000000,68.000000,3328.000000,231.000000,237.000000,168.000000,2552.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3851.000000,360.000000,62.000000,1397.000000,597.000000,7082.000000,254.000000,254.000000,253.000000,7112.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [28]:
scaler = StandardScaler() # we are using standard scaler in this problem. 
# MinMax scaler and standard scaler are the common schemes for the scaling. 
covtype_Xtrain_scale = scaler.fit_transform(covtype_Xtrain)
covtype_Xtest_scale = scaler.transform(covtype_Xtest)

In [33]:
covtype_Xtrain_df = pd.DataFrame(covtype_Xtrain_scale)
covtype_Xtrain_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
count,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,...,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05
mean,-2.398341e-16,-2.474928e-17,1.027882e-16,1.228750e-16,-2.406137e-17,-1.029105e-16,-5.247795e-16,-5.242750e-16,-3.349025e-16,5.975598e-17,...,-5.220431e-18,-9.115500e-17,1.900145e-17,-2.659898e-17,-2.060656e-17,1.039500e-18,1.834412e-19,2.838753e-17,7.031913e-18,5.014060e-17
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,...,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-3.933573e+00,-1.391704e+00,-1.883508e+00,-1.267975e+00,-3.650420e+00,-1.506995e+00,-7.923916e+00,-1.130743e+01,-3.720984e+00,-1.494739e+00,...,-2.146959e-01,-3.159133e-01,-2.902529e-01,-5.289803e-02,-5.737349e-02,-1.466930e-02,-2.115883e-02,-1.662407e-01,-1.559070e-01,-1.235729e-01
25%,-5.381767e-01,-8.732315e-01,-6.812617e-01,-7.595373e-01,-6.774128e-01,-7.978761e-01,-5.292395e-01,-5.234052e-01,-6.141002e-01,-7.216994e-01,...,-2.146959e-01,-3.159133e-01,-2.902529e-01,-5.289803e-02,-5.737349e-02,-1.466930e-02,-2.115883e-02,-1.662407e-01,-1.559070e-01,-1.235729e-01
50%,1.301802e-01,-2.564280e-01,-1.469300e-01,-2.416838e-01,-2.821575e-01,-2.259645e-01,2.176975e-01,1.347744e-01,1.249825e-02,-2.038234e-01,...,-2.146959e-01,-3.159133e-01,-2.902529e-01,-5.289803e-02,-5.737349e-02,-1.466930e-02,-2.115883e-02,-1.662407e-01,-1.559070e-01,-1.235729e-01
75%,7.270552e-01,9.324831e-01,5.209845e-01,5.398043e-01,3.880581e-01,6.267737e-01,7.032065e-01,6.916957e-01,6.652049e-01,4.303104e-01,...,-2.146959e-01,-3.159133e-01,-2.902529e-01,-5.289803e-02,-5.737349e-02,-1.466930e-02,-2.115883e-02,-1.662407e-01,-1.559070e-01,-1.235729e-01
max,3.211056e+00,1.826401e+00,6.932964e+00,5.275810e+00,9.530486e+00,3.056116e+00,1.562184e+00,1.552392e+00,2.910516e+00,3.920311e+00,...,4.657751e+00,3.165425e+00,3.445271e+00,1.890430e+01,1.742965e+01,6.816957e+01,4.726160e+01,6.015374e+00,6.414081e+00,8.092390e+00


In [34]:
# 20 News group data
# This data should be classified into news categories.
newsgroup = fetch_20newsgroups()
print(newsgroup.DESCR)

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality

In [42]:
# We need more pre processing for the text data.
print(len(newsgroup.target_names)) 

20
